In [149]:
import os
import json
import pandas as pd
import traceback

In [150]:
from langchain import HuggingFaceHub

In [151]:
#from dotenv import load_dotenv

#load_dotenv()  # take environment variables from .env.

In [152]:
# os.getenv("HUGGINGFACE_API_KEY")
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''

In [153]:
#llm=HuggingFaceHub(repo_id="google/flan-t5-large",task="text-generation",model_kwargs={"temperature": 0.1})  

In [154]:
#llm

In [155]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [156]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [157]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [158]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [159]:
quiz_chain=LLMChain(llm=HuggingFaceHub(repo_id="google/gemma-7b",model_kwargs={
    "temperature": 0.6, "max_length":50}), prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [160]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [161]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [162]:
review_chain=LLMChain(llm=HuggingFaceHub(repo_id="google/gemma-7b",model_kwargs={
    "temperature": 0.6,"max_length":50}), prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [163]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [164]:
file_path="/home/phoenix/Downloads/mcqgenai(env)/data.txt"

In [165]:
file_path

'/home/phoenix/Downloads/mcqgenai(env)/data.txt'

In [166]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [167]:
print(TEXT)

Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model.[1][2] A prompt is natural language text describing the task that an AI should perform.[3]

A prompt for a text-to-text language model can be a query such as "what is Fermat's little theorem?",[4] a command such as "write a poem about leaves falling",[5] or a longer statement including context, instructions,[6] and conversation history. Prompt engineering may involve phrasing a query, specifying a style,[5] providing relevant context[7] or assigning a role to the AI such as "Act as a native French speaker".[8] A prompt may include a few examples for a model to learn from, such as asking the model to complete "maison → house, chat → cat, chien →" (the expected response being dog),[9] an approach called few-shot learning.[10]

When communicating with a text-to-image or a text-to-audio model, a typical prompt is a description of a desired output such as "a high-quality pho

In [168]:
# Serialize the Python dictionary into a JSON-formatted string
json_formate=json.dumps(RESPONSE_JSON)
json_formate

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [169]:
# from langchain.evaluation import JsonEqualityEvaluator

# evaluator = JsonEqualityEvaluator()
# # Equivalently
# # evaluator = load_evaluator("json_equality")
# result = evaluator.evaluate_strings(prediction='{"a": 1}', reference='{"a": 1}')
# print(result)

In [170]:
from langchain.evaluation import JsonEqualityEvaluator

In [171]:
NUMBER=5 
SUBJECT="prompt engineering"
TONE="simple"


In [172]:
evaluator = JsonValidityEvaluator()

response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json_formate,
        }
        )

result = evaluator.evaluate_strings(prediction=response , reference= json_formate)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model.[1][2] A prompt is natural language text describing the task that an AI should perform.[3]

A prompt for a text-to-text language model can be a query such as "what is Fermat's little theorem?",[4] a command such as "write a poem about leaves falling",[5] or a longer statement including context, instructions,[6] and conversation history. Prompt engineering may involve phrasing a query, specifying a style,[5] providing relevant context[7] or assigning a role to the AI such as "Act as a native French speaker".[8] A prompt may include a few examples for a model to learn from, such as asking the model to complete "maison → house, chat → cat, chien →" (the expected response being dog),[9] an approach called few-shot learning.[10]

When communicating with a text-to-image

/home/phoenix/Downloads/mcqgenai(env)/env/lib/python3.12/site-packages/langchain/evaluation/schema.py:128: UserWarning: Ignoring reference in JsonValidityEvaluator, as it is not expected.
  warn(self._skip_reference_warning)


In [177]:
result

{'score': 0, 'reasoning': "expected string or bytes-like object, got 'dict'"}

In [178]:
response

{'text': 'Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model.[1][2] A prompt is natural language text describing the task that an AI should perform.[3]\n\nA prompt for a text-to-text language model can be a query such as "what is Fermat\'s little theorem?",[4] a command such as "write a poem about leaves falling",[5] or a longer statement including context, instructions,[6] and conversation history. Prompt engineering may involve phrasing a query, specifying a style,[5] providing relevant context[7] or assigning a role to the AI such as "Act as a native French speaker".[8] A prompt may include a few examples for a model to learn from, such as asking the model to complete "maison → house, chat → cat, chien →" (the expected response being dog),[9] an approach called few-shot learning.[10]\n\nWhen communicating with a text-to-image or a text-to-audio model, a typical prompt is a description of a desired output such as "a h

In [179]:
quiz=response.get("quiz")
print(quiz)


Text:Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model.[1][2] A prompt is natural language text describing the task that an AI should perform.[3]

A prompt for a text-to-text language model can be a query such as "what is Fermat's little theorem?",[4] a command such as "write a poem about leaves falling",[5] or a longer statement including context, instructions,[6] and conversation history. Prompt engineering may involve phrasing a query, specifying a style,[5] providing relevant context[7] or assigning a role to the AI such as "Act as a native French speaker".[8] A prompt may include a few examples for a model to learn from, such as asking the model to complete "maison → house, chat → cat, chien →" (the expected response being dog),[9] an approach called few-shot learning.[10]

When communicating with a text-to-image or a text-to-audio model, a typical prompt is a description of a desired output such as "a high-quali

In [180]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning1.csv",index=False)

In [ ]:
# from datetime import datetime
# datetime.now().strftime('%m_%d_%Y_%H_%M_%S')